In [11]:
import sys
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.utils import shuffle
warnings.filterwarnings('ignore')
import tensorflow as tf

TRAIN_PATH = "../../data/train.csv"
TEST_PATH = "../../data/test.csv"

def get_train_data():
    data = pd.read_csv(TRAIN_PATH)
    return data

data = get_train_data().head(20000)

In [12]:
#カテゴリ変数のダミー化
cat = data.filter(regex="cat").columns
bin = data.filter(regex="bin").columns
cat = cat.append(bin)
non_cat = list(set(data.columns) - set(cat))

for tag in cat:
    data[tag] = data[tag].astype(str)
data = pd.concat((data, pd.get_dummies(data[cat])), axis=1).drop(cat,axis=1)

#欠損値の存在するnon_cat値
for tag in non_cat:
    data[tag][data[tag]==-1] = data[tag][data[tag]!=-1].mean()
    
#不要な行を削除しつつnp.arrayに変換
TEACHER = np.c_[data.target.values]
data = data.drop(['id', 'target'], axis=1)
INPUT = data.values
#del data

In [19]:
#移植しただけ
class DNN:
    def __init__(self, n_in, n_hids):
        self.n_in = n_in
        self.n_hids = n_hids
        self.weights = []
        self.biases = []
        self._x = None
        self._t = None
        self._keep_prob = None
        self._sess = None
        self._history = {'accuracy':[], 'loss':[]}
        self.Accuracy = None
    
    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)
    
    def bias_variable(self, shape):
        initial = tf.zeros(shape)
        return tf.Variable(initial)
    
    def inference(self, x, keep_prob):
        for i, n_hid in enumerate(self.n_hids):
            if i == 0:
                input = x
                input_dim = self.n_in
            else:
                input = output
                input_dim = self.n_hids[i-1]
            
            self.weights.append(self.weight_variable([input_dim, n_hid]))
            self.biases.append(self.bias_variable([n_hid]))
            
            #activation function
            h = tf.nn.relu(tf.matmul(input, self.weights[-1]) + self.biases[-1])
            output = tf.nn.dropout(h, keep_prob)
            
        #1消した
        self.weights.append(self.weight_variable([self.n_hids[-1],1]))
        self.biases.append(self.bias_variable([1]))
        
        #activation function
        return tf.nn.relu(tf.matmul(output, self.weights[-1]) + self.biases[-1])
    
    def loss(self, y, t):
        cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1-t) * tf.log(1-y))
        return cross_entropy
    
    def training(self, loss):
        return tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    def accuracy(self, y, t):
        correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)
        return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def fit(self, X_train, Y_train, epochs=100, batch_size=100, p_keep=0.5):
        x = tf.placeholder(tf.float32, shape=[None, self.n_in])
        t = tf.placeholder(tf.float32, shape=[None,1])
        keep_prob = tf.placeholder(tf.float32)
        
        self._x = x
        self._t = t
        self._keep_prob = keep_prob
        
        y = self.inference(x, keep_prob)
        loss = self.loss(y, t)
        train_step = self.training(loss)
        self.Accuracy = self.accuracy(y, t)
        
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        self._sess = sess
        
        N_train = len(X_train)
        n_batches = N_train // batch_size
        
        for epoch in range(epochs):
            X_, Y_ = shuffle(X_train, Y_train)
            
            for i in range(n_batches):
                start = i * batch_size
                end = start + batch_size
                sess.run(train_step, feed_dict={x:X_[start:end], t:Y_[start:end], keep_prob:p_keep})
                loss_ = loss.eval(session=sess, feed_dict={x:X_train, t:Y_train, keep_prob:1.0})
                accuracy_ = self.Accuracy.eval(session=sess, feed_dict={x:X_train, t:Y_train, keep_prob:1.0})
                
                self._history['loss'].append(loss_)
                self._history['accuracy'].append(accuracy_)
                
                print('epoch:', epoch,'loss:', loss_,'accuracy:', accuracy_)
        return self._history
        
    def evaluate(self, X_test, Y_test):
        return self.Accuracy.eval(session=self._sess,
                    feed_dict={self._x:X_test, self._t:Y_test, self._keep_prob:1.0})

In [20]:
model = DNN(n_in=243, n_hids=[100,100])
model.fit(INPUT, TEACHER, epochs=10, batch_size=20, p_keep=0.5)
indices = np.random.permutation(range(len(value)))[:10000]
print('accuracy:', model.evaluate(INPUT[indices], TEACHER[indices]))

epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265
epoch: 0 loss: nan accuracy: 0.96265


KeyboardInterrupt: 